In [6]:
import torch
from torch import nn
import os
import random
from torch.utils import data
from tqdm import tqdm
import numpy as np
from copy import deepcopy
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
import warnings
import csv
import re

warnings.filterwarnings('ignore')

global extracted_grads

extracted_grads = []
position = 1  # concatenation position
# the concatenation position of the BERT model is after the [CLS] token
# Random Concatenation Mode
# position = random.randint(1,500)

BERT_path = 'PreTrainedModelBert'  # path to bert model
tokenize = BertTokenizer.from_pretrained(os.path.join(BERT_path, 'vocab.txt'))
model_config = BertConfig.from_pretrained(os.path.join(BERT_path, 'config.json'))
Model = BertForSequenceClassification.from_pretrained(os.path.join(BERT_path, 'pytorch_model.bin'), config=model_config)

# Load model related information

# Print the number of Total Parameters
# total = [param.nelement() for param in Model.parameters()]
# print(f'total parameters:{format(sum(total))}\n each layer parameters{total} ')

'''
SST-2 Data
'''


### Load data

def read_sst_data(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            data.append(line[1])
            labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels


def read_sst_test_data(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '0':  # neg
                data.append(line[1])
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels


def load_sst_array(data_arrays, batch_size, is_train=True):
    """Constructs a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


def load_sst_data(batch_size, num_steps=500):
    train_data = read_sst_data("SST-2/train.tsv")
    test_data = read_sst_test_data("SST-2/test.tsv")
    train_encoding = tokenize(train_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    train_iter = load_sst_array(
        (train_encoding['input_ids'], train_encoding['token_type_ids'], torch.tensor(train_data[1])),
        batch_size)
    test_iter = load_sst_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        1,
        is_train=False)
    return train_iter, test_iter

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]


### Train

def train(net, train_iter, lr, num_epochs, device):
    print('---------------------------start---------------------')
    optimizer = torch.optim.AdamW(net.parameters(), lr=lr)
    net = net.to(device[0])
    for epoch in range(num_epochs):
        net.train()
        print(f' epoch {epoch + 1}')
        train_losses = []
        train_accs = []
        train_length = 0
        for batch in tqdm(train_iter):
            a, b, y = batch
            a = a.to(device[0])
            b = b.to(device[0])
            y = y.to(device[0])
            outputs = net(input_ids=a, token_type_ids=b, labels=y)
            logits = outputs.logits
            l = outputs.loss
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            acc = (logits.argmax(dim=-1) == y).float().mean()
            train_losses.append(l)
            train_accs.append(acc)
            train_length += len(y)
        print("Learning rate for epoch %d：%f" % (epoch + 1, optimizer.param_groups[0]['lr']))
        train_loss = sum(train_losses) / len(train_iter)
        train_acc = sum(train_accs) / len(train_iter)
        print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}   acc = {train_acc:.5f}")
    print('Training process has finished.')
    print('the loss of model {:.3f}'.format(train_loss))


def evaluate_no(net, test_iter):
    net = net.to(device[0])
    net.eval()
    valid_accs = []
    with torch.no_grad():
        for batch in tqdm(test_iter):
            a, b, y = batch
            a = a.to(device[0])
            b = b.to(device[0])
            y = y.to(device[0])
            outputs = net(input_ids=a, token_type_ids=b, labels=y)
            acc = (outputs.logits.argmax(dim=-1) == y).float().mean()
            valid_accs.append(acc)
    valid_acc = sum(valid_accs) / len(test_iter)
    return valid_acc


### Trigger Token

def init_trigger_tokens(trigger, num_trigger_tokens):
    # Initialize trigger tokens, we use 'the' as initial trigger token
    trigger_token_ids = [1996] * num_trigger_tokens  # 1996 means 'the'
    trigger_token_tensor = torch.tensor(trigger_token_ids)
    return trigger_token_tensor


def evaluate(net, test_iter, trigger_token_tensor):
    # evaluate the accuracy of the model after concatenating the initial trigger token
    net = net.to(device[0])
    net.eval()
    valid_accs = []
    n = torch.tensor([0] * len(trigger_token_tensor))
    m = deepcopy(trigger_token_tensor)
    m = m.unsqueeze(0)
    n = n.unsqueeze(0)
    with torch.no_grad():
        for batch in tqdm(test_iter):
            a, b, y = batch
            a = torch.cat((a[:, :position], m.repeat_interleave(a.shape[0], dim=0), a[:, position:]), dim=1)
            b = torch.cat((b[:, :position], n.repeat_interleave(b.shape[0], dim=0), b[:, position:]), dim=1)
            a = a.to(device[0])
            b = b.to(device[0])
            y = y.to(device[0])
            outputs = net(input_ids=a, token_type_ids=b, labels=y)
            acc = (outputs.logits.argmax(dim=-1) == y).float().mean()
            valid_accs.append(acc)
    valid_acc = sum(valid_accs) / len(test_iter)
    return valid_acc


def extract_grad_hook(net, grad_in, grad_out):  # store the gradient in extracted_grads
    extracted_grads.append(grad_out[0].mean(dim=0))


def add_hook(net):
    for module in net.modules():
        if isinstance(module, nn.Embedding):
            hook = module.register_backward_hook(extract_grad_hook)
            break
    return hook


def get_gradient(net, test_iter, trigger_token_tensor):  # Calculate the loss to get the gradient
    net = net.to(device[0])
    net.train()
    m = deepcopy(trigger_token_tensor)
    m = m.unsqueeze(0)
    n = torch.tensor([0] * len(trigger_token_tensor))
    n = n.unsqueeze(0)
    optimizer = torch.optim.AdamW(net.parameters())
    for batch in tqdm(test_iter):
        a, b, y = batch
        a = torch.cat((a[:, :position], m.repeat_interleave(a.shape[0], dim=0), a[:, position:]), dim=1)
        b = torch.cat((b[:, :position], n.repeat_interleave(b.shape[0], dim=0), b[:, position:]), dim=1)
        a = a.to(device[0])
        b = b.to(device[0])
        y = y.to(device[0])
        outputs = net(input_ids=a, token_type_ids=b, labels=y)
        l = outputs.loss
        optimizer.zero_grad()
        l.backward()


def process_gradient(length, num_trigger_tokens):  # Process the gradient to get the average gradient
    extracted_grads_copy = extracted_grads
    extracted_grads_copy[0] = extracted_grads_copy[0]
    temp = extracted_grads_copy[0]
    temp = temp.unsqueeze(0)
    for i in range(1, length - 1):
        extracted_grads_copy[i] = extracted_grads_copy[i]
        extracted_grads_copy[i] = extracted_grads_copy[i].unsqueeze(0)
        temp = torch.cat((temp, extracted_grads_copy[i]), dim=0)
    average_grad = temp.mean(dim=0)[position:position + num_trigger_tokens]
    return average_grad


def hotflip_attack(averaged_grad, embedding_matrix,
                   num_candidates=1, increase_loss=False):
    averaged_grad = averaged_grad.cpu()
    embedding_matrix = embedding_matrix.cpu()
    averaged_grad = averaged_grad.unsqueeze(0)
    gradient_dot_embedding_matrix = torch.einsum("bij,kj->bik",
                                                 (averaged_grad, embedding_matrix))
    if not increase_loss:
        gradient_dot_embedding_matrix *= -1
        # lower versus increase the class probability.
    if num_candidates > 1:  # get top k options
        _, best_k_ids = torch.topk(gradient_dot_embedding_matrix, num_candidates, dim=2)
        return best_k_ids.detach().cpu().numpy()[0]  # Return candidates
    _, best_at_each_step = gradient_dot_embedding_matrix.max(2)
    return best_at_each_step[0].detach().cpu().numpy()


def collection_attack(net, test_iter, num_candidates, num_epoch, trigger='the',  # Summarize each function
                      num_trigger_tokens=3):
    trigger_token_tensor = init_trigger_tokens(trigger, num_trigger_tokens)
    print(f'Concatenation location:{position}')
    valid_acc = evaluate(net, test_iter, trigger_token_tensor)
    print(f'Initial trigger tokens state：the accuracy {valid_acc:.5f}')
    embedding_weight = get_embedding_weight(net)
    for i in range(num_epoch):
        extracted_grads.clear()
        hook = add_hook(net)
        get_gradient(net, test_iter, trigger_token_tensor)
        hook.remove()
        average_grad = process_gradient(len(test_iter), num_trigger_tokens)
        hot_token = hotflip_attack(average_grad, embedding_weight, num_candidates, increase_loss=True)
        hot_token_tensor = torch.from_numpy(hot_token)
        trigger_token_tensor, valid_acc = select_best_candid(net, test_iter, hot_token_tensor, trigger_token_tensor,
                                                             valid_acc)
        print(f'after {i + 1} rounds of attacking\ntriggers: {trigger_token_tensor} \nthe accuracy :{valid_acc:.5f} ')
    return trigger_token_tensor, valid_acc  # Return the final trigger tokens (trigger length) and the accuracy after the attack


def get_embedding_weight(net):
    for module in net.modules():
        if isinstance(module, nn.Embedding):
            weight = module.weight
            break
    return weight


def select_best_candid(net, test_iter, candid_trigger, trigger_token, valid_acc):
    # Concatenate each candidate to each input to determine the final trigger token
    n = torch.tensor([0] * len(trigger_token))
    n = n.unsqueeze(0)
    trigger_token = trigger_token.unsqueeze(0)
    net.eval()
    valid_accs = []
    for i in range(candid_trigger.shape[0]):
        trigger_token_temp = deepcopy(trigger_token)
        for j in range(candid_trigger.shape[1]):
            trigger_token_temp[0, i] = candid_trigger[i, j]
            valid_accs = []
            for batch in tqdm(test_iter):
                a, b, y = batch
                a = torch.cat((a[:, :position], trigger_token_temp.repeat_interleave(a.shape[0], dim=0),
                               a[:, position:]), dim=1)
                b = torch.cat((b[:, :position], n.repeat_interleave(b.shape[0], dim=0),
                               b[:, position:]), dim=1)
                a = a.to(device[0])
                b = b.to(device[0])
                y = y.to(device[0])
                outputs = net(input_ids=a, token_type_ids=b, labels=y)
                acc = (outputs.logits.argmax(dim=-1) == y).float().mean()
                valid_accs.append(acc)
            temp = sum(valid_accs) / len(test_iter)
            if temp < valid_acc:
                valid_acc = temp
                trigger_token[0, i] = candid_trigger[i, j]
    return trigger_token[0], valid_acc  # Return the final trigger token and the accuracy after the attack


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at PreTrainedModelBert/pytorch_model.bin and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_iter, test_iter = load_sst_data(10)
# Data preprocessing and loading
print("reading data finished\n")
print(len(train_iter))
print(len(test_iter))

reading data finished

6735
912


In [8]:
device = try_all_gpus()
# device = [torch.device('cpu')]
train(Model, train_iter, 5e-6, 3, device)  # base BERT

---------------------------start---------------------
 epoch 1


100%|██████████| 6735/6735 [03:44<00:00, 29.94it/s]


Learning rate for epoch 1：0.000005
[ Train | 001/003 ] loss = 0.23149   acc = 0.90634
 epoch 2


100%|██████████| 6735/6735 [03:51<00:00, 29.06it/s]


Learning rate for epoch 2：0.000005
[ Train | 002/003 ] loss = 0.12508   acc = 0.95656
 epoch 3


100%|██████████| 6735/6735 [03:50<00:00, 29.16it/s]


Learning rate for epoch 3：0.000005
[ Train | 003/003 ] loss = 0.08985   acc = 0.96975
Training process has finished.
the loss of model 0.090


In [9]:
torch.save(Model, 'Bert_sst.bin')

In [10]:
evaluate_no(Model, test_iter)

100%|██████████| 912/912 [00:05<00:00, 181.18it/s]


tensor(0.9068, device='cuda:0')

In [11]:
model = torch.load('Bert_sst.bin')
evaluate_no(model, test_iter)

100%|██████████| 912/912 [00:04<00:00, 184.49it/s]


tensor(0.9068, device='cuda:0')

In [13]:
collection_attack(model, test_iter, 5, 10, trigger='the', num_trigger_tokens=3)

Concatenation location:1


100%|██████████| 912/912 [00:05<00:00, 164.78it/s]


Initial trigger tokens state：the accuracy 0.91009


100%|██████████| 912/912 [00:06<00:00, 137.49it/s]


after 1 rounds of attacking
triggers: tensor([20570,  1996, 21735]) 
the accuracy :0.89254 


...

100%|██████████| 912/912 [00:07<00:00, 130.25it/s]


after 9 rounds of attacking
triggers: tensor([15299, 17950, 27149]) 
the accuracy :0.01645 


100%|██████████| 912/912 [00:06<00:00, 140.19it/s]

after 10 rounds of attacking
triggers: tensor([15299, 17950, 27149]) 
the accuracy :0.01645 


(tensor([15299, 17950, 27149]), tensor(0.0164, device='cuda:0'))